# 🔺 Construcción de la Triangulación Causal
Este notebook toma un árbol de Galton-Watson con espina y genera la triangulación causal correspondiente en el cilindro \( S \times [n, n+1] \).

In [ ]:
!pip install numpy matplotlib networkx

In [ ]:

import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

# Simulación básica del árbol para tener vértices por niveles
def sample_rhok():
    while True:
        k = np.random.geometric(p=0.5)
        if k > 0 and np.random.rand() < (k / (2 ** (k + 1))) / (1 / (2 ** k)):
            return k

def sample_pk():
    r = np.random.rand()
    k = 0
    while r > 1 / (2 ** (k + 1)):
        r -= 1 / (2 ** (k + 1))
        k += 1
    return k

def generate_layers(n_layers=5):
    layers = [[0]]  # raíz
    spine_index = [0]
    node_id = 1
    for _ in range(n_layers):
        new_layer = []
        new_spine = []
        for idx, parent in enumerate(layers[-1]):
            if parent == spine_index[-1]:
                k = sample_rhok()
            else:
                k = sample_pk()
            children = list(range(node_id, node_id + k))
            node_id += k
            new_layer.extend(children)
            if parent == spine_index[-1] and k > 0:
                new_spine.append(children[np.random.randint(0, k)])
        spine_index.append(new_spine[0] if new_spine else -1)
        layers.append(new_layer)
    return layers, spine_index

def triangulate_layers(layers):
    G = nx.Graph()
    for i, layer in enumerate(layers):
        for v in layer:
            G.add_node((i, v), layer=i)
        if i < len(layers) - 1:
            upper = layers[i + 1]
            for j in range(len(upper)):
                G.add_edge((i, layer[j % len(layer)]), (i + 1, upper[j]))
            for j in range(len(upper) - 1):
                G.add_edge((i + 1, upper[j]), (i + 1, upper[j + 1]))
    return G

def draw_triangulation(G):
    pos = {}
    for (layer, v) in G.nodes:
        pos[(layer, v)] = (v, -layer)
    nx.draw(G, pos, with_labels=False, node_size=100, node_color='skyblue')
    plt.title("Triangulación Causal (esquemática)")
    plt.show()


In [ ]:

layers, spine = generate_layers(n_layers=6)
G = triangulate_layers(layers)
draw_triangulation(G)
